In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [40]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [41]:
df_train = pd.read_csv(r"E:\Machine learning project\titanic/train.csv")
df_test = pd.read_csv(r"E:\Machine learning project\titanic/test.csv")

relevent_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

# Handeel missing data

imputer = SimpleImputer(strategy = 'most_frequent')

df_train[relevent_features] = imputer.fit_transform(df_train[relevent_features])
df_test[relevent_features] = imputer.fit_transform(df_test[relevent_features])

# Encode catagorical variable as numaric

df_train['Sex'] = df_train['Sex'].map({'male' : 1, 'female' : 0})
df_test['Sex'] = df_test['Sex'].map({'male' : 1, 'female' : 0})

df_train['Embarked'] = df_train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
df_test['Embarked'] = df_test['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})



# Spliting data into two catagories

x_train = df_train[relevent_features]
y_train = df_train['Survived']
x_test = df_test[relevent_features]










In [42]:
# Scaledown calues

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [54]:
import pickle
pickle.dump(scaler,open('scaling.pkl','wb'))

In [43]:
## Spliting data
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 33)

In [44]:
## finding best model using GridSearchCV

model_params = {
    'svm' : {
        'model' : svm.SVC(),
        'params' : {
            'C': [0.1, 1, 10], 
            'kernel': ['linear', 'poly', 'rbf'], 
            'gamma': [0.01, 0.1, 1]
        }
   },
    
    'random_forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'min_samples_split': [2, 5, 10],
            'max_features': ['sqrt', 'log2']
        }
    },
    
    'logistic_regression' : {
        'model' : LogisticRegression(),
        'params' : {
            'C': [0.1, 1, 10], 
            'penalty': ['l2'], 
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'], 
            'max_iter': [100, 500, 1000]
        }
    },
    
    'gaussin_nb' : {
        'model' : GaussianNB(),
        'params' : {
            'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
        }
    },
    
    'knn' : {
        'model' : KNeighborsClassifier(),
        'params' : {
            'n_neighbors': [3, 5, 7], 
            'weights': ['uniform', 'distance'], 
            'metric': ['euclidean', 'manhattan']
        }
    }
}

In [45]:
score = []

for model_name, mp in model_params.items():
    
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train)
    score.append({
        'model' : model_name,
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_
    })

In [46]:
df_score = pd.DataFrame(score, columns = ['model', 'best_score', 'best_params'])
df_score

,model,best_score,best_params
0,svm,0.820221,"{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}"
1,random_forest,0.821639,"{'max_depth': 3, 'max_features': 'sqrt', 'min_..."
2,logistic_regression,0.801950,"{'C': 0.1, 'max_iter': 100, 'penalty': 'l2', '..."
3,gaussin_nb,0.782261,{'var_smoothing': 1e-09}
4,knn,0.806166,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."


In [47]:
# Fit model to SVM after Grid Search

svm_model = svm.SVC(C=1, gamma=0.1, kernel='rbf')
svm_model.fit(x_train, y_train)

SVC(C=1, gamma=0.1)

In [48]:
y_pred = svm_model.predict(x_val)

In [49]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.92      0.86       107
           1       0.85      0.69      0.76        72

    accuracy                           0.83       179
   macro avg       0.83      0.81      0.81       179
weighted avg       0.83      0.83      0.82       179



In [50]:
y_pred_final = svm_model.predict(x_test)

In [51]:
##output = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': y_pred_final})
##output.to_csv('submission.csv', index=False)

In [52]:
pickle.dump(svm_model, open('titanic_model0.1.0.pkl', 'wb'))

In [53]:
pickled_model = pickle.load(open('titanic_model0.1.0.pkl', 'rb'))